In [206]:
import torch, torch.utils.data
from torch import nn
import torchvision, torchvision.datasets
from torchvision import transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import List

from fedlab.main.fedlab_benchmarks.leaf.dataloader import get_LEAF_dataloader

from torch.profiler import profile, record_function, ProfilerActivity
import time
from sklearn.metrics import f1_score   
import numpy as np
    
import random


In [207]:
print(torch.__version__)




2.0.1+cu117


In [208]:
def get_femnist_shakespeare_dataset(**args):
    if args['dataset'] == 'femnist' or args['dataset'] == 'shakespeare':
        trainloader, testloader = get_LEAF_dataloader(dataset=args['dataset'],
                                                      client_id=args['rank'])
    else:
        raise ValueError("Invalid dataset:", args['dataset'])
    return trainloader, testloader

In [209]:
train_loaders = []
test_loaders = []
num_nodes = 5

for i in range(num_nodes):
    args = {'dataset': 'femnist', 'rank': i}
    trainloader, testloader = get_femnist_shakespeare_dataset(**args)
    train_loaders.append(trainloader)
    test_loaders.append(testloader)

#loader = zip(train_loaders, test_loaders)



In [210]:
print(len(train_loaders))
print(len(test_loaders))
print(test_loaders)
print(len(train_loaders[0].dataset))

5
5
[<torch.utils.data.dataloader.DataLoader object at 0x00000271DCC609D0>, <torch.utils.data.dataloader.DataLoader object at 0x00000271DCE0B430>, <torch.utils.data.dataloader.DataLoader object at 0x00000271DCE0A5C0>, <torch.utils.data.dataloader.DataLoader object at 0x00000271DCE0A110>, <torch.utils.data.dataloader.DataLoader object at 0x00000271DCE09060>]
343


In [211]:
device = torch.device('cuda')
torch.cuda.get_device_name(device)

'NVIDIA GeForce RTX 3070 Ti Laptop GPU'

In [212]:
transform = transforms.Compose([
    transforms.ToTensor(),
])
target_transform = lambda x: torch.nn.functional.one_hot(torch.tensor(x),62).float()

In [213]:
# def split_dataset(dataset, n):
#     torch.utils.data.random_split(dataset, )

In [214]:
# train_dataset2 = torchvision.datasets.MNIST('datasets/', download=True, train=True, transform=transform, target_transform=target_transform),
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1000000, num_workers=0)

# test_dataset = torchvision.datasets.MNIST('datasets/', download=True, train=False, transform=transform, target_transform=target_transform)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000000, num_workers=0)

'''
# Inital MNIST
train_datasets = torch.utils.data.random_split(torchvision.datasets.MNIST('datasets/',
                                                                           download=True, train=True, transform=transform, target_transform=target_transform), [1/num_nodes]*num_nodes)
train_loaders = [torch.utils.data.DataLoader(i, batch_size=1000000, num_workers=0) for i in train_datasets]
'''

"\n# Inital MNIST\ntrain_datasets = torch.utils.data.random_split(torchvision.datasets.MNIST('datasets/',\n                                                                           download=True, train=True, transform=transform, target_transform=target_transform), [1/num_nodes]*num_nodes)\ntrain_loaders = [torch.utils.data.DataLoader(i, batch_size=1000000, num_workers=0) for i in train_datasets]\n"

In [215]:

x_i_j = {}
s_i = {}
w_k = {}

In [216]:
class FederatedNode:
    def __init__(self, train_dataloader, test_dataloader, node_id) -> None:
        self.dataset_size = len(train_dataloader.dataset)
        channels_sum, channels_squared_sum, num_batches = 0, 0, 0
        for data, _ in train_dataloader:
            # Mean over batch, height and width, but not over the channels
            channels_sum += torch.mean(data, dim=[0])
            channels_squared_sum += torch.mean(data**2, dim=[0])
            num_batches += 1
    
        self.mean = channels_sum / num_batches
        self.test_loss = 0.0
        self.test_accuracy = 0.0
        self.train_accuracy = 0.0
        self.f1_score = 0.0


        # std = sqrt(E[X^2] - (E[X])^2)
        self.std = (channels_squared_sum / num_batches - self.mean ** 2) ** 0.5
        print("Dataset size ", self.dataset_size)
        print("Mean size ", self.mean.shape)
        print("Standard deviation size ", self.std.shape)
        
        
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784,56),
            nn.ReLU(),
            nn.Linear(56, 62)
        ).to(device)

        '''
        only_digits=False
        self.network = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3),
        nn.MaxPool2d(2, stride=2),
        nn.Conv2d(32, 64, kernel_size=3),
        nn.Dropout(0.25),
        nn.Flatten(),
        nn.Linear(9216, 128),
        nn.Dropout(0.5),
        nn.Linear(128, 10 if only_digits else 62),
        nn.ReLU()
        ).to(device)
        '''
        self.optimizer = torch.optim.Adam(self.network.parameters())
        self.criterion = nn.CrossEntropyLoss()
        self.train_dataloader = train_dataloader
        self.test_dataloader = test_dataloader
        self.node_id = node_id
    
    def train_epoch(self):
        losses = []
        for idx, (data_x, data_y) in enumerate(self.train_dataloader):
            #print("data_x", data_x.shape)
            #print("data_y", data_y.shape)
            #plt.imshow(data_x[1][0])
            output = self.network(data_x.to(device))
            self.optimizer.zero_grad()
            #print(output.shape)
            #print(data_y)
            data_y = target_transform(data_y)
            
            loss = nn.functional.mse_loss(output, data_y.to(device))
            losses.append(loss.item())
            loss.backward()
            self.optimizer.step()
        return sum(losses)/len(losses)
    
    def testing(self):
        correct = 0
        total = 0
        # test_running_loss = 0.0
        predictions = []
        labels = []
        test_accuracy = []
        test_running_losses = []
        for idx, (data_x, data_y) in enumerate(self.test_dataloader):
            output = self.network(data_x.to(device))
            
            _, predicted = torch.max(output.data, 1)
            data_y = data_y.to(predicted.device)
            total += data_y.size(0)
            correct += (predicted == data_y).sum().item()
            
            loss = self.criterion(output, data_y)
            predictions.append(predicted.cpu())
            labels.append(data_y.cpu())
            
            test_running_losses.append(loss.item())
            #print('Epoch %d test loss: %.3f' % (idx + 1, test_running_losses[-1]))
            #print(test_running_losses)
            #test_running_loss += loss.item()
            #test_loss.append(test_running_loss / len(self.test_dataloader))
            test_accuracy.append(100 * correct / total)
        
        #print(predictions)
        predictions = np.concatenate(predictions)
        labels = np.concatenate(labels)
        self.f1_score= f1_score(labels, predictions, average="weighted")
        self.test_accuracy = test_accuracy[-1]
        #print(self.test_accuracy)
        self.test_loss =  sum(test_running_losses)/len(test_running_losses)
        #print("test loss ",self.test_loss)
       
    def share_x_ij(self):
        x = self.network.state_dict()
        r = torch.randint(1, 5, [num_nodes])
        s_r = torch.sum(r)
        j=self.node_id
        for i in range(num_nodes):
            x_i_j[(i,j)]={}
            for key in x.keys():
                x_i_j[(i,j)][key]=x[key]*r[i]/s_r
                
        
    
    def share_s_i(self):
        i = self.node_id
        s_i[i] = {}
        for key in x_i_j[(0,0)].keys():
            s_i[i][key] = sum([x_i_j[(i,j)][key] for j in range(num_nodes)])

    def share_x_ij_neighbours(self, neighbour_nodes):
        j=self.node_id
        num_neighbour_nodes = len(neighbour_nodes)
        x = self.network.state_dict()
        r = torch.randint(1, 5, [num_neighbour_nodes])
        s_r = torch.sum(r)
        
        for i, n in enumerate(neighbour_nodes):
            x_i_j[(n,j)]={}
            for key in x.keys():
                x_i_j[(n,j)][key]=x[key]*r[i]/s_r*self.dataset_size

    def share_s_i_neighbours(self, neighbour_nodes):
        i = self.node_id
        s_i[i] = {}
        
        for key in x_i_j[list(x_i_j.keys())[0]].keys():
            s_i[i][key] = sum([x_i_j[(i,j)][key] for j in neighbour_nodes])


    def share_FedavgP2P(self, nodes):
        dataset_size_all = [n.dataset_size for n in nodes]
        i = self.node_id
        C = 0.5
        A = num_nodes - 1
        m = C*A
        #n_k = self.dataset_size

        # Random clients
        S_t = random.sample([ j for j in range(num_nodes) if j!=i], int(m)) 
        S_t.append(self.node_id)
        #print("Clients ",S_t)
        #S_t = random.sample(list(range(num_nodes)) , int(m))
        #print("n_k ",n_k)
        #print("m neighbors", m)
        #print("S_t", S_t)
        w_c = self.network.state_dict()
        n_c = self.dataset_size
        n_t = n_c
        for s in S_t:
            n_t += dataset_size_all[s]
        
        for j in S_t: 
            nodes[j].share_x_ij_neighbours(S_t)
        for j in S_t: 
            nodes[j].share_s_i_neighbours(S_t)
                
        for key in nodes[0].network.state_dict():
            w_c[key] = w_c[key]*n_c/n_t

            for j in S_t: 
                n_k = dataset_size_all[j]
                #states = nodes[j].network.state_dict()
                averaged_state = {}
            
                for key in s_i[list(s_i.keys())[0]].keys():
                    param_value = 0
                    for i in s_i:
                        param_value += s_i[i][key]
                    
                    averaged_state[key] = param_value/len(S_t)/n_t
                #print(averaged_state)
                #contributed_state = n_k/n_t*averaged_state[key]
                #w_c[key] = w_c[key] + contributed_state
                
        self.network.load_state_dict(w_c)
           

        # for key in x_i_j[(0,0)].keys():
        #     for j in S_t:      
        #         n_k = dataset_size_all[j]
        #         contributed_weight = n_k/n_t*x_i_j[(i, j)][key]
        #         w_k[i][key] = w_k[i].setdefault(key, 0) + contributed_weight
                

                #w_k[i][key] = sum( * [x_i_j[(i, j)][key] for j in S_t])
            
        #print(w_k)

        #print("Total dataset samples n_t", n_t)    
        
    def preprocessFedavg(self, dataset_size_all, nodes):
        i = self.node_id
        C = 0.5
        A = num_nodes - 1
        m = C*A
        #n_k = self.dataset_size

        # Random clients
        S_t = random.sample([ j for j in range(num_nodes) if j!=i], int(m)) 
        S_t.append(self.node_id)
        #print("Clients ",S_t)

        w_c = self.network.state_dict()
        n_c = self.dataset_size
        n_t = n_c
        for s in S_t:
            n_t += dataset_size_all[s]
        
        return S_t
                

              
             







In [217]:
#for i, loader_data in enumerate(zip(*loader)):
#    train_loaders, test_loaders = loader_data

nodes = []
for i in range(num_nodes):
    nodes.append(FederatedNode(train_loaders[i], test_loaders[i], i)) 

#nodes = [FederatedNode(l, i) for i,l in enumerate())]

Dataset size  343
Mean size  torch.Size([1, 28, 28])
Standard deviation size  torch.Size([1, 28, 28])
Dataset size  372
Mean size  torch.Size([1, 28, 28])
Standard deviation size  torch.Size([1, 28, 28])
Dataset size  364
Mean size  torch.Size([1, 28, 28])
Standard deviation size  torch.Size([1, 28, 28])
Dataset size  358
Mean size  torch.Size([1, 28, 28])
Standard deviation size  torch.Size([1, 28, 28])
Dataset size  284
Mean size  torch.Size([1, 28, 28])
Standard deviation size  torch.Size([1, 28, 28])


In [218]:
losses = {}
#with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
#    with record_function("model_training"):

start_time = time.time()
start_time = time.time()
dataset_size_all = {}
epochs ={i:0 for i in range(num_nodes)}
epoch_total = 50
for i in tqdm(range(epoch_total*num_nodes)):
    cands = [i for i in epochs if epochs[i]<epoch_total]
    node = nodes[cands[torch.randint(len(cands), (1,)).item()]]
    del cands
    loss = node.train_epoch()
    node.share_FedavgP2P(nodes)
    
    epochs[node.node_id] += 1
    losses[(node.node_id, epochs[node.node_id])]=loss
    # x_i_j = {}
    # s_i = {}
    # for node in nodes:
    #     node.share_x_ij()
    
    # for node in nodes:
    #     node.share_s_i()
    
    # FedavgP2P


    #states = [i.network.state_dict() for i in nodes]
    
    # av_state = {}
    
    # for key in states[0]:
    #     av_state[key] = sum([s[key] for s in states])/num_nodes
        
        
      
    #av_state = {}
    #print(s_i[0])
    #for key in s_i[0].keys():
    #    av_state[key]=torch.mean(torch.tensor([s_i[i][key] for i in s_i]), dim=[0])

'''
 # Averaging weights after SMPC logic   
    averaged_state = {}
    for key in s_i[0].keys():
        param_value = 0
        for i in s_i:
            param_value += s_i[i][key]
        
        averaged_state[key] = param_value/num_nodes

    for node in nodes:
        node.network.load_state_dict(averaged_state)

'''
    
    
    # states = [i.network.state_dict() for i in nodes]
    # av_state = {}
    # for key in states[0]:
    #     av_state[key] = sum([s[key] for s in states])/num_nodes
    # for node in nodes:
    #     node.network.load_state_dict(av_state)

time_FL_SMPC = time.time() - start_time

print("Time taken by FL with SMPC", time_FL_SMPC)
    


  0%|          | 0/250 [00:00<?, ?it/s]C:\Users\ankit\AppData\Local\Temp\ipykernel_20060\3311650527.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_transform = lambda x: torch.nn.functional.one_hot(torch.tensor(x),62).float()
100%|██████████| 250/250 [00:03<00:00, 63.51it/s]

Time taken by FL with SMPC 3.9382474422454834


In [219]:
losses

{(1, 1): 0.07353825867176056,
 (3, 1): 0.06238468115528425,
 (4, 1): 0.04898064645628134,
 (2, 1): 0.04796862415969372,
 (2, 2): 0.016456063836812973,
 (3, 2): 0.016480324789881706,
 (1, 2): 0.016471896320581436,
 (1, 3): 0.016134091342488926,
 (1, 4): 0.01611370158692201,
 (3, 3): 0.01613425773878892,
 (0, 1): 0.047261593863368034,
 (4, 2): 0.016511892278989155,
 (0, 2): 0.01646471892793973,
 (4, 3): 0.016149551918109257,
 (0, 3): 0.016124846413731575,
 (4, 4): 0.01611912250518799,
 (2, 3): 0.01613809789220492,
 (3, 4): 0.016115390385190647,
 (4, 5): 0.01611354077855746,
 (2, 4): 0.01611182528237502,
 (1, 5): 0.01611176257332166,
 (4, 6): 0.01611120191713174,
 (4, 7): 0.016109831631183624,
 (0, 4): 0.016089300935467083,
 (3, 5): 0.016113239650925,
 (3, 6): 0.016110874091585476,
 (3, 7): 0.016109076639016468,
 (4, 8): 0.016108769923448563,
 (4, 9): 0.016108093162377674,
 (1, 6): 0.016110006719827652,
 (1, 7): 0.016108502944310505,
 (4, 10): 0.01610735182960828,
 (2, 5): 0.0161061938852

[[0.047261593863368034, 0.07353825867176056, 0.04796862415969372, 0.06238468115528425, 0.04898064645628134], [0.01646471892793973, 0.016471896320581436, 0.016456063836812973, 0.016480324789881706, 0.016511892278989155], [0.016124846413731575, 0.016134091342488926, 0.01613809789220492, 0.01613425773878892, 0.016149551918109257], [0.016089300935467083, 0.01611370158692201, 0.01611182528237502, 0.016115390385190647, 0.01611912250518799], [0.01607208512723446, 0.01611176257332166, 0.016106193885207176, 0.016113239650925, 0.01611354077855746], [0.016055598855018616, 0.016110006719827652, 0.016103843227028847, 0.016110874091585476, 0.01611120191713174], [0.01604357734322548, 0.016108502944310505, 0.01610296592116356, 0.016109076639016468, 0.016109831631183624], [0.01603916970392068, 0.01610778955121835, 0.016101155430078506, 0.016107644885778427, 0.016108769923448563], [0.016034527371327083, 0.016106045494476955, 0.016100449487566948, 0.016107162460684776, 0.016108093162377674], [0.016025815

In [221]:
losses2 = [[losses[(j,i)] for j in range(num_nodes) ] for i in range(1,epoch_total+1)]
print(losses2)
plt.plot(losses2)

In [222]:
start_time = time.time()
for i in tqdm(range(5)):
    epoch_losses = []
    
    for node in nodes:
        loss = node.train_epoch()
        epoch_losses.append(loss)
    losses.append(epoch_losses)


    
    av_state = {}
    
    
    
    states = [i.network.state_dict() for i in nodes]
    av_state = {}
    for key in states[0]:
        av_state[key] = sum([s[key] for s in states])/num_nodes
    for node in nodes:
        node.network.load_state_dict(av_state)

time_FL = time.time() - start_time

print("Time taken by FL without SMPC", time_FL)
    

  0%|          | 0/5 [00:00<?, ?it/s]C:\Users\ankit\AppData\Local\Temp\ipykernel_20060\3311650527.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_transform = lambda x: torch.nn.functional.one_hot(torch.tensor(x),62).float()
  0%|          | 0/5 [00:00<?, ?it/s]


AttributeError: 'dict' object has no attribute 'append'

In [ ]:
#print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

In [ ]:
global_mean = 0.0
global_std = 0.0
eqn1= 0.0 
eqn2 = 0.0
eqn3 = 0.0
for node in nodes:
    eqn1 += node.mean * node.dataset_size
    eqn2 += node.dataset_size 
    eqn3 += node.std


global_mean = eqn1/eqn2
global_std = eqn3/eqn2
print("Global Mean", global_mean.shape)
print("Global std ", global_std.shape)
#plt.imshow(global_mean.squeeze())
#plt.imshow(global_std.squeeze())

Global Mean torch.Size([1, 28, 28])
Global std  torch.Size([1, 28, 28])


In [ ]:
#nodes = [FederatedNode(l, i) for i,l in enumerate(test_loaders)]

test_losses = []
with torch.no_grad():
    for i, node in enumerate(nodes):
        print()
        print("Node ", i)
        node.testing()
        #test_losses.append(node.test_loss)
        print("Node Test Accuracy", node.test_accuracy)
        print("Node Test loss", node.test_loss)
        print("Node F1_Score ", node.f1_score)

        




Node  0
Node Test Accuracy 15.384615384615385
Node Test loss 4.100019454956055
Node F1_Score  0.04102564102564103

Node  1
Node Test Accuracy 7.142857142857143
Node Test loss 4.109908580780029
Node F1_Score  0.009523809523809525

Node  2
Node Test Accuracy 9.75609756097561
Node Test loss 4.109867095947266
Node F1_Score  0.017344173441734414

Node  3
Node Test Accuracy 5.0
Node Test loss 4.10900354385376
Node F1_Score  0.0047619047619047615

Node  4
Node Test Accuracy 6.25
Node Test loss 4.108822345733643
Node F1_Score  0.007352941176470588
